# 제네레이터 사용하기

제네레이터는 전통적인 언어와 파이썬을 구분 짓는 또 다른 특징적인 기능

배울 것

1. 제네레이터의 이론적 근거, 소개 배경.

2. 제네레이터를 통한 문제 해결법

3. 제네레이터(또는 이터러블)을 파이썬스럽게 구현할 수 있는법

이를 통해 왜 언어에서 자동으로 반복을 지원하게 되는지, 어떻게 코루틴이나 비동기 프로그래밍 같은 기능을 지원하기 위한 기본 기능이 되었는지 살펴 보기

## 제네레이터 만들기

아주 오래전 2001년 부터 소개

제네레이터는 한 번에 하나씩 구성요소를 반환하는 이터러블을 생성하는 객체.

거대한 요소를 한꺼번에 메모리에 저장하는 대신 특정 요소를 어떻게 만드는지 아는 객체를 만들어서 필요할 때마다 하나씩만 가져오는 것

-> 게으른 연산을 통해 메모리를 절약하여 무거운 객체 사용 가능. 무한 시퀀스도 가능 

### 제네레이터 개요

목표: 대규모의 구매 기록에서 최저 판매가, 최고 판매가, 평균 판매가를 계산하기

-> 간단하게 for 문을 통해서 루프마다 각 지표를 업데이트 하는 방식의 구현

In [2]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()

    def _initialize():
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self > max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    def __str__(self):
        return (
            f"{self.__class__.__name}({self.min_price},"
            f"{self.max_price, {self.avg_price}})"
        )

이제 이 모든 정보를 로드해서 담아서 반환해주는 함수를 만들어 보기 

### 리스트에 불러오기

In [5]:
def _load_purchases(file_name):
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))
            
    return purchases

정확한 결과를 반환하지만, 파일에서 모든 정보를 읽어 list에 저장하기 떄문에
로드하는데 시간이 오래 걸리며 메모리 용량을 넘을 수도 있다.

결과는 정확함. 하지만 _load_purchases에서 다 읽어와 list에 저장하고
다 읽어온 데이터를 다시 for문으로 돌리기 때문에 비효율적

* load해서 list에 저장하는 단계에서 시간과 메모리 둘 다 많이 필요함.

* 앞에서 계산하는 코드는 한번에 하나의 데이터만을 사용함

-> 굳이 모든 데이터를 모두 읽어 메모리에 보관할 이유 x

해결책: 제네레이터를 만들어 필요한 값을 그때 그때 만들기

또한 앞에서 계산하는 코드는 한 번에 하나의 데이터만을 사용하고 있다.
그러므로 굳이 모든 데이터를 한 번에 모두 읽어 메모리에 보관할 이유가 없다.

해결책은 제네레이터를 만드는 것이다. 파일의 전체 내용을 리스트에 보관하는 대신
필요한 값을 그때 그때 가져오는 것이다. 다음과 같이 코드를 수정한다.

### 제네레이터로 효율적

In [33]:
def load_purchases(file_name):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

메모리를 많이 필요하던 리스트 사라지고 return도 사라짐

이때의 load_purchases 함수를 제네레이터 함수, 또는 단순히 제네레이터라고 함.

-> 파이썬에선 어떤 함수라도 yield 키워드를 쓰면 제네레이터 함수가 된다.

yield가 포함된 이 함수를 호출하면 제네레이터의 인스턴스를 만듬

In [35]:
load_purchases("purchases.csv")

<generator object load_purchases at 0x000001A888D422C8>

모든 제네레이터 객체는 이터러블입니다. (이터러블은 뒤에서 더 다룹니다.)

여기서 중요한 것은 제네레이터로 바꿨지만 

함수의 사용 코드가 **그대로** 라는것!

즉 이터러블을 사용하면 for 루프의 다형성을 보장하는 강력한 추상화가 가능함!

-> 이터러블 인터페이스를 따르면 투명하게 객체의 요소를 반복하는 것이 가능하다.

In [ ]:
#

In [36]:
list(enumerate("abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [37]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start
    def next(self):
        current = self.current
        self.current += 1 
        return current

In [38]:
seq = NumberSequence()
seq.next()

0

In [39]:
seq.next()

1

In [40]:
seq2 = NumberSequence(10)

In [41]:
seq2.next()

10

In [42]:
seq2.next()

11

In [43]:
list(zip(NumberSequence(), "abcdef"))

TypeError: zip argument #1 must support iteration

In [60]:
class SequenceOfNumbers_1:
    
    def __init__(self, start=0):
        self.current = start
        
    def __next__(self):
        current = self.current
        self.current += 1
        return current


In [61]:
a = SequenceOfNumbers_1()

In [63]:
1 in a

TypeError: argument of type 'SequenceOfNumbers_1' is not iterable

In [59]:
list(zip(SequenceOfNumbers_1(),"abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [44]:
class SequenceOfNumbers:
    
    def __init__(self, start=0):
        self.current = start
        
    def __next__(self):
        current = self.current
        self.current += 1
        return current
    
    def __iter__(self):
        return self 

## test data 만들기

In [ ]:
"""Helper to generate test data."""
import os
from tempfile import gettempdir

PURCHASES_FILE = os.path.join("./", "purchases.csv")


def create_purchases_file(filename, entries=1_000_000):
    if os.path.exists(PURCHASES_FILE):
        return

    with open(filename, "w+") as f:
        for i in range(entries):
            line = f"2018-01-01,{i}\n"
            f.write(line)



create_purchases_file(PURCHASES_FILE)